<a href="https://colab.research.google.com/github/EP111111/Langchain_reflection/blob/main/Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install google-cloud-aiplatform langchain pandas datasets google-api-python-client


In [5]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [2]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [1]:
from google.cloud import aiplatform
from langchain.chat_models import ChatVertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.schema import HumanMessage, SystemMessage
print(f"Vertex AI SDK version: {aiplatform.__version__}")

Vertex AI SDK version: 1.31.0


In [2]:
import vertexai
PROJECT_ID = "ics-analysis-dev"
vertexai.init(project=PROJECT_ID, location="us-central1")

In [3]:
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=512,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

## 1. langchain -prompt template

In [13]:
from langchain import PromptTemplate, LLMChain

def reflection(des,text):
  template = """Description: {des}

  Answer: Based on above pattern description and example cases above, determine if the pattern  is presented in the following text:
  “{text_for_evaluation}”

  Provide a response comprising a result and an explanation for why the pattern is or is not present in JSON format. The explanation should be around 30 words.

  Example output:

  {{  "pattern presented" : “Yes” or “No”,
    “Explanation”: “The explanation goes here”
  }}"""

  prompt = PromptTemplate(template=template,input_variables=["des","text_for_evaluation"])
  feed=prompt.format(des=des,text_for_evaluation=text)
  return llm(feed)

In [14]:
des="""Consider a pattern named "Outcome experience" that may or may not be present in some text.

The text is provided in response to the following question: “Briefly outline the experience. Make a brief note about whether it was good or bad or in between.”

Here is a description of  "Outcome experience":
“Instead of reflecting on a process, the "experience" is an outcome (e.g. I got a bad test result).”

Now consider a few cases where the pattern is present. In the following cases, the text is given and then the rationale for why the pattern was identified follows:

Case 1:
Text: “when doing my aim questions on evolution, I was having trouble finding good questions to ask right away”
Rationale: “The student has used the outcome of them having trouble finding good questions to ask right away, which is likely secondary to many possible causes such as cognitive load intolerance, lack of practice with the AIM step or lack of focus. Thus, their reflection is scattered and unfocused. Factors that would suggest NO outcome as experience:
Factors that would suggest NO outcome as experience:
N/A

Case 2:
Text: “Briefly outline the experience. Make a brief note about whether it was good or bad or in between.I noticed I didn't finish the Kolb's I started a few days ago”
Rationale: “Student’s experience is focused on a result and has not mentioned anything in regards to a process in their experience.

Factors that would suggest NO outcome as experience:
Section “If you struggled, at what points? Were there any triggers?”, student have expanded on their process to some extent.
Factors that ULTIMATELY suggest outcome as experience:
Any sections where the student does reflect more, it is limited in its depth – a natural consequence of outcome as experience.
”

Case 3:
Text: “I recently failed a computer science test, am struggling with maths and overall with uni and feel burnt out. Frustrating experience (Try to have this kolbs done in 1 hour)”
Rationale: “High indication of outcome as experience. Kolb’s seems to have been used as a way to vent and not as a tool to reflect on processes.
Factors that would suggest NO outcome as experience:
n/a, the student definitely has an outcome as experience.

Factors that ULTIMATELY suggest outcome as experience:
See above comments.”

Case 4:
Text: “I struggled to draw the mind map, the visualisations were messy and the diagram lacked logicalcoherence. This was negative.”
Rationale: “The student has used the outcome of them having struggling to draw the mindmap in their reflection, but takes this to mean reflecting on the process of them doing the mindmap. This is only borderline outcome as experience.

Factors that would suggest NO outcome as experience:
The reflection is metacognitive
They focus on the mindmapping process

Factors that ULTIMATELY suggest outcome as experience:
Their experience could be more explicit in stating that they “did a mindmap” rather than the outcome of poor visualisation etc.
”"""
text="I struggled to draw the mind map, the visualisations were messy and the diagram lacked logical coherence. This was negative."


In [15]:
feedback=reflection(des,text)
print(feedback)

{ "pattern presented" : "Yes",
   "Explanation" : "The student has used the outcome of them having struggling to draw the mindmap in their reflection, but takes this to mean reflecting on the process of them doing the mindmap. This is only borderline outcome as experience." 
  }


## 2. ChatVertexAI with langchain

In [16]:
from langchain.chat_models import ChatVertexAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage

In [20]:
def chat_models(des,text):
  chat = ChatVertexAI()
  template = ChatPromptTemplate.from_messages([
    ("system", des),
    ("human", """Based on above pattern description and example cases above, determine if the pattern  is presented in the following text:
  “{text_for_evaluation}”

  Provide a response comprising a result and an explanation for why the pattern is or is not present in JSON format. The explanation should be around 30 words.

  Example output:

  {{  "pattern presented" : “Yes” or “No”,
    “Explanation”: “The explanation goes here”
  }}"""),
    ])


  messages = template.format_messages(
    text_for_evaluation=text
  )
  return chat(messages)


In [22]:
feed1=chat_models(des,text)
print(feed1.content)

 {
  "pattern presented": "Yes",
  "Explanation": "The student has used the outcome of them having struggling to draw the mindmap in their reflection, but takes this to mean reflecting on the process of them doing the mindmap."
}
